Import the Python ODBC library and pandas

In [2]:
#!pip install pyodbc
import pyodbc
import pandas.io.sql
import pandas as pd
import numpy as np

You will need to set up a connection to your local database server. Just enter your local server and database name.

In [4]:
local_server="POWER\\POWER17" #"TARDIS\\TARDIS"
local_database ="SASClone" #"ScholarshipAwardingProcess"
sqlConnection = pyodbc.connect('DRIVER={SQL Server};server=' + local_server + ';database=' + local_database + ';trusted_connection=true')
sqlConnection.autocommit=True;

Verify that the connection is working by pulling the algorithm table.

In [5]:
sql="Select * from Algorithms"
algorithms = pandas.io.sql.read_sql(sql, sqlConnection)
algorithms

,AlgorithmId,AlgorithmName,AlgorithmDescription
0,1,Merit Only,Only use top applicant for scholarship
1,2,Merit Only Disqualify After Exceeding Maximum,Use top but top out a single person
2,3,Merit Only Awarding Can’t Exceed Maximum\r\n,Use top but don't let go over certain amount
3,4,Maximum One Award Per Applicant,Only allow an applicant to have a single schol...
4,5,Split with all qualified applicants\r\n,Split award with all qualified applicants
5,6,Split with minimum qualified applicants\r\n,Split award with applicants up to a certain nu...
6,7,Split with minimum amount given\r\n,Split evenly up to split doesn't bring award a...


In [6]:
new_awarding_group=pandas.io.sql.read_sql("CreateAwardingGroup 'FromPython'", sqlConnection)
new_awarding_group["AwardingGroupId"][0]


3

In [7]:
new_awarding_group_id=new_awarding_group["AwardingGroupId"][0]
pandas.io.sql.read_sql("Select * from awardinggroups", sqlConnection)

,AwardingGroupId,AwardingGroupName
0,1,Paper Demo
1,2,Paper Denormalized Demo
2,3,FromPython


In [8]:
def GetAnalysis (awarding_group_id,maximum_award,minimum_award,max_applicants,run_analysis_first):
    sql="GetAnalysis " + str(awarding_group_id) + ", "  + str(maximum_award) + ", "
    sql =sql  + str(minimum_award) + "," + str(max_applicants) + "," + str(run_analysis_first)
    return pandas.io.sql.read_sql(sql, sqlConnection)

GetAnalysis(1,1500,130,2,1)

,ScholarshipAwardAnalysisId,AwardingGroupId,AlgorithmId,MaximumAward,MinimumAward,MaxApplicants,RA1,RA2,RA3,NumberOfAwarded,UniqueAwardees,MaximumAwarded,MinimumAwarded
0,5,1,5,1500.0,130.0,2,True,True,True,6,3,1375.0,375.0
1,6,1,6,1500.0,130.0,2,True,True,True,6,3,1375.0,375.0
2,7,1,7,1500.0,130.0,2,True,True,True,6,3,1375.0,375.0
3,1,1,1,1500.0,130.0,2,True,True,True,3,1,2750.0,2750.0
4,3,1,3,1500.0,130.0,2,False,True,True,3,3,1000.0,750.0
5,4,1,4,1500.0,130.0,2,False,True,True,3,3,1000.0,750.0
6,2,1,2,1500.0,130.0,2,False,True,False,3,2,2000.0,750.0


In [9]:
def InsertDenormalizedData(awarding_group_id,scholarshipName,scholarshipAmount,applicantName, applicantRank):
    sql="InsertIntoDenormalizedEntry " + str(awarding_group_id) + ","
    sql=sql + "'"+ str(scholarshipName) + "',"
    sql=sql +   str(scholarshipAmount) + ","
    sql=sql + "'" + str(applicantName) + "',"
    sql=sql + str(applicantRank)
    #return sql
    return pandas.io.sql.read_sql(sql, sqlConnection)

InsertDenormalizedData(new_awarding_group_id,"s1",1000.00,"a2",2)


,DenormalizedEntryId,AwardingGroupId,Scholarship,ScholarshipAward,Applicant,ApplicantRanking
0,10,3,s1,1000.0,a2,2


In [11]:
!pip install xlrd
from pandas import ExcelWriter
from pandas import ExcelFile
df = pd.read_excel('Example Data/DemoData.xlsx', sheetname='Sheet1')
for i in df.index:
    scholarship=df["Scholarship"][i]
    scholarship_award=df["ScholarshipAward"][i]
    applicant=df["Applicant"][i]
    applicant_ranking =df["ApplicantRanking"][i]
    InsertDenormalizedData(new_awarding_group_id, scholarship,scholarship_award,applicant, applicant_ranking)
 


C:\ProgramData\Anaconda34\envs\HwPy36\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [12]:
pandas.io.sql.read_sql("SELECT * FROM dbo.DenormalizedEntries WHERE AwardingGroupId =" + str(new_awarding_group_id),sqlConnection)

,DenormalizedEntryId,AwardingGroupId,Scholarship,ScholarshipAward,Applicant,ApplicantRanking
0,10,3,s1,1000.0,a2,2
1,11,3,SC1,1000.0,A1,1
2,12,3,SC1,1000.0,A2,2
3,13,3,SC1,1000.0,A4,4
4,14,3,SC2,750.0,A1,1
5,15,3,SC2,750.0,A3,3
6,16,3,SC3,500.0,A1,1
7,17,3,SC3,500.0,A2,2
8,18,3,SC3,500.0,A3,3
9,19,3,SC3,500.0,A4,4


In [13]:
def create_analysis_from_spreadsheet(awarding_group_name,filePath,maximum_award,minimum_award,max_applicants,run_analysis_first):
    new_awarding_group=pandas.io.sql.read_sql("CreateAwardingGroup 'FromPython'", sqlConnection)
    new_awarding_group_id=new_awarding_group["AwardingGroupId"][0]
    from pandas import ExcelWriter
    from pandas import ExcelFile
    df = pd.read_excel(filePath, sheetname='Sheet1')
    for i in df.index:
        scholarship=df["Scholarship"][i]
        scholarship_award=df["ScholarshipAward"][i]
        applicant=df["Applicant"][i]
        applicant_ranking =df["ApplicantRanking"][i]
        InsertDenormalizedData(new_awarding_group_id, scholarship,scholarship_award,applicant, applicant_ranking)
    return (GetAnalysis(new_awarding_group_id,maximum_award,minimum_award,max_applicants,run_analysis_first))
create_analysis_from_spreadsheet("test case","Example Data/DemoData.xlsx",1500,130,2,1)

C:\ProgramData\Anaconda34\envs\HwPy36\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


,DenormalizedEntryAnalysisId,AwardingGroupId,AlgorithmId,MaximumAward,MinimumAward,MaxApplicants,RA1,RA2,RA3,NumberOfAwarded,UniqueAwardees,MaximumAwarded,MiniumumAwarded
0,6,4,6,1500.0,130.0,2,True,True,True,6,3,1125.00,375.00
1,2,4,2,1500.0,130.0,2,True,True,True,3,2,1750.00,500.00
2,1,4,1,1500.0,130.0,2,True,True,True,3,1,2250.00,2250.00
3,3,4,3,1500.0,130.0,2,False,True,False,3,2,1500.00,750.00
4,5,4,5,1500.0,130.0,2,False,False,True,9,4,833.33,458.33
5,7,4,7,1500.0,130.0,2,False,False,True,8,4,875.00,333.33
6,4,4,4,1500.0,130.0,2,False,False,True,3,3,1000.00,500.00


In [15]:
def GetScholarshipAwards(algorithm_id,  awading_group_id, maximum_award,minimum_award,max_applicants):
    sql="GetScholarshipAwards " + str(algorithm_id) + ","
    sql=sql +   str(awading_group_id) + ","
    sql=sql +   str(maximum_award) + ","
    sql=sql +   str(minimum_award) + ","
    sql=sql + str(max_applicants)
    #return sql
    return pandas.io.sql.read_sql(sql, sqlConnection)
GetScholarshipAwards(1,1,1500.0,130.0,2)

,ScholarshipAwardId,AwardingGroupId,ScholarshipName,FirstName,LastName,AlgorithmId,Award
0,1,1,SC1,A1,A1,1,1000.0
1,2,1,SC1,A1,A1,1,1000.0
2,3,1,SC2,A1,A1,1,750.0


In [18]:
def GetDeonormalizedScholarshipAwards(algorithm_id,  awading_group_id, maximum_award,minimum_award,max_applicants):
    sql="GetDeonormalizedScholarshipAwards " + str(algorithm_id) + ","
    sql=sql +   str(awading_group_id) + ","
    sql=sql +   str(maximum_award) + ","
    sql=sql +   str(minimum_award) + ","
    sql=sql + str(max_applicants)
    #return sql
    return pandas.io.sql.read_sql(sql, sqlConnection)
GetScholarshipAwards(4,2,1500.0,130.0,2)

,ScholarshipAwardId,AwardingGroupId,ScholarshipName,FirstName,LastName,AlgorithmId,Award
